In [1]:
import torch
from cnnf.model_cifar import WideResNet
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from utils import *
from tqdm import tqdm
from train import train_adv, test, test_pgd

In [2]:
#args
class Arg:
    pass
#opt params
args = Arg()
args.batch_size = 256
args.test_batch_size = 128
args.epochs = 500
args.lr = 0.05
args.power = 0.9
args.momentum = 0.9
args.wd = 5e-4
args.grad_clip = True
args.dataset = 'cifar10'
args.schedule = 'poly'
args.no_cuda = True
args.seed = 1
args.log_interval = 400

#adver training params
args.eps = 0.063
args.eps_iter = 0.02
args.nb_iter = 7
args.clean = 'supclean'

#hyper params
args.mse_parameter = 0.1
args.clean_parameter = 0.05
args.res_parameter = 0.1

#model params
args.layers = 40
args.widen_factor = 2
args.droprate = 0.0
args.ind = 5
args.max_cycles = 2
args.save_model = 'CNNF_superes_cifar'
args.model_dir = 'models'

In [3]:
#params cuda
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [4]:
#transforms
train_transform_cifar = transforms.Compose(
  [transforms.RandomHorizontalFlip(),
   transforms.RandomCrop(32, padding=4),
   transforms.ToTensor(),
   transforms.Normalize([0.5] * 3, [0.5] * 3)])

test_transform_cifar = transforms.Compose(
  [transforms.ToTensor(),
   transforms.Normalize([0.5] * 3, [0.5] * 3)])

In [5]:
#model and dataset
train_data = datasets.CIFAR10(
    'data', train=True, transform=train_transform_cifar, download=True)
test_data = datasets.CIFAR10(
    'data', train=False, transform=test_transform_cifar, download=True)
train_loader = torch.utils.data.DataLoader(
  train_data, batch_size=args.batch_size,
  shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
  test_data, batch_size=args.test_batch_size,
  shuffle=True, num_workers=4, pin_memory=True)
num_classes = 10
model = WideResNet(args.layers, 10, args.widen_factor, args.droprate, args.ind, args.max_cycles, args.res_parameter).to(device)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
#optimizer and scheduler
optimizer = torch.optim.SGD(
      model.parameters(),
      args.lr,
      momentum=args.momentum,
      weight_decay=args.wd)

scheduler = torch.optim.lr_scheduler.LambdaLR(
  optimizer, lr_lambda=lambda step: lr_poly(1.0, step, args.epochs * len(train_loader), args.power))

In [7]:
# Begin training
best_acc = 0

for epoch in tqdm(range(args.epochs)):
    train_loss, train_acc = train_adv(args, model, device, train_loader, optimizer, scheduler, epoch,
      cycles=args.max_cycles, mse_parameter=args.mse_parameter, clean_parameter=args.clean_parameter, clean=args.clean)

    test_loss, test_acc = test(args, model, device, test_loader, cycles=args.max_cycles, epoch=epoch)

    print(f"Epoch {epoch}:")
    print('loss', 'train:', train_loss)
    print('acc', 'train:', train_acc)
    print('loss', 'test:', test_loss)
    print('acc', 'test:', test_acc)

    # Save the model with the best accuracy
    if test_acc > best_acc and args.save_model is not None:
        best_acc = test_acc
        experiment_fn = args.save_model
        torch.save(model.state_dict(),
                   args.model_dir + "/{}-best.pt".format(experiment_fn))

    if ((epoch+1)%50)==0 and args.save_model is not None:
        experiment_fn = args.save_model
        torch.save(model.state_dict(),
                   args.model_dir + "/{}-epoch{}.pt".format(experiment_fn,epoch))
        pgd_acc = test_pgd(args, model, device, test_loader, epsilon=args.eps)

        print('pgd_acc', 'test:', pgd_acc)

# Save final model
if args.save_model is not None:
    experiment_fn = args.save_model
    torch.save(model.state_dict(),
               args.model_dir + "/{}.pt".format(experiment_fn))

  0%|          | 0/500 [00:13<?, ?it/s]


AssertionError: Torch not compiled with CUDA enabled